In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from PIL import Image
import pickle
import ipywidgets as widgets
from IPython.display import display
import os
import io
import seaborn as sns
from ipyevents import Event # conda install -c conda-forge ipyevents

%matplotlib osx 
# figures will be popups

In [2]:
workdir = "eg_data/"
os.listdir(workdir)

['img_20190122_ICB_s1_p1_r1_a1_ac_31RD_S100__1.tiff',
 'img_20190122_ICB_s1_p1_r1_a1_ac_31RD_CD3__1.tiff',
 'img_20190122_ICB_s1_p1_r1_a1_ac_31RD_CD68__1.tiff',
 'pixeltype.p',
 'img_20190122_ICB_s1_p1_r1_a1_ac_31RD_CD4__1.tiff',
 'img_20190122_ICB_s1_p1_r1_a1_ac_31RD_CD45RO__1.tiff',
 'img_20190122_ICB_s1_p1_r1_a1_ac_31RD_CD8a__1.tiff',
 'img_20190122_ICB_s1_p1_r1_a1_ac_31RD_CD20__1.tiff',
 'img_20190122_ICB_s1_p1_r1_a1_ac_31RD_SOX10__1.tiff']

# Input

In [3]:
 # list of all cores names
fn_ls = [ "img_20190122_ICB_s1_p1_r1_a1_ac_31RD"]

# dictionary in the form {core name : integer pixel type map } 
pixeltype_dict = {fn_ls[0]: pickle.load(open(workdir + 'pixeltype.p', 'rb'))['pixeltype'] }

# number of pixel types (excluding 0)
N_type = 9

# potential channels you want to inspect. 
markers = ['CD3','CD4','CD8a','CD20','CD45RO','CD68','S100','SOX10']



In [4]:
# a function to load/construct single channel images as np array. (it should read the entire image)
def singlechannel(fn, m, **kargs):
    return np.array(Image.open(f"{workdir}/{fn}_{m}__1.tiff"))

## (optional) pixel type annotation
It does't really matter since the color code can be modified in the GUI.

In [5]:
# list of types, in the order of pixel type integers.
types = [f'type {i}' for i in np.arange(1, N_type + 1)]
types

['type 1',
 'type 2',
 'type 3',
 'type 4',
 'type 5',
 'type 6',
 'type 7',
 'type 8',
 'type 9']

In [6]:
# replace generic types to more specific names 
types[1] = 'B'
types[7] = 'Macro'

## (optional) initial color code
It does't really matter since the color code can be modified in the GUI.

In [7]:
# default color palette
palette = sns.color_palette('Paired',9) # maxium 12 colors
palette

[(0.6509803921568628, 0.807843137254902, 0.8901960784313725),
 (0.12156862745098039, 0.47058823529411764, 0.7058823529411765),
 (0.6980392156862745, 0.8745098039215686, 0.5411764705882353),
 (0.2, 0.6274509803921569, 0.17254901960784313),
 (0.984313725490196, 0.6039215686274509, 0.6),
 (0.8901960784313725, 0.10196078431372549, 0.10980392156862745),
 (0.9921568627450981, 0.7490196078431373, 0.43529411764705883),
 (1.0, 0.4980392156862745, 0.0),
 (0.792156862745098, 0.6980392156862745, 0.8392156862745098)]

In [8]:
# or, build a custom palette
allcolors = sns.color_palette('Paired')
allcolors

[(0.6509803921568628, 0.807843137254902, 0.8901960784313725),
 (0.12156862745098039, 0.47058823529411764, 0.7058823529411765),
 (0.6980392156862745, 0.8745098039215686, 0.5411764705882353),
 (0.2, 0.6274509803921569, 0.17254901960784313),
 (0.984313725490196, 0.6039215686274509, 0.6),
 (0.8901960784313725, 0.10196078431372549, 0.10980392156862745),
 (0.9921568627450981, 0.7490196078431373, 0.43529411764705883),
 (1.0, 0.4980392156862745, 0.0),
 (0.792156862745098, 0.6980392156862745, 0.8392156862745098),
 (0.41568627450980394, 0.23921568627450981, 0.6039215686274509),
 (1.0, 1.0, 0.6),
 (0.6941176470588235, 0.34901960784313724, 0.1568627450980392)]

In [9]:
# continueing building custom palette
palette[4] = allcolors[9]
palette[8] = (1, 1, 0) # (r, g, b) in range (0,1)
palette

[(0.6509803921568628, 0.807843137254902, 0.8901960784313725),
 (0.12156862745098039, 0.47058823529411764, 0.7058823529411765),
 (0.6980392156862745, 0.8745098039215686, 0.5411764705882353),
 (0.2, 0.6274509803921569, 0.17254901960784313),
 (0.41568627450980394, 0.23921568627450981, 0.6039215686274509),
 (0.8901960784313725, 0.10196078431372549, 0.10980392156862745),
 (0.9921568627450981, 0.7490196078431373, 0.43529411764705883),
 (1.0, 0.4980392156862745, 0.0),
 (1, 1, 0)]

## !! 

So far you should have 6 variables: fn_ls, pixeltype_dict, N_types, types, markers, and palette.

And a function to load the single channel images for local inspection.

Below is the GUI part and shouldn't need modifications.

***You should press "refresh" after changing cores or color code.***


In [21]:
refreshrate = 500 # time intervals for event listeners im milisecond
initial_padding = 30 # initital value for regional view


# some color conversion functions 
def rgb_to_hex_i(r,g,b):
    ''' integer rgb to hex'''
    return '#%02x%02x%02x' % (r,g,b)

def rgb_to_hex_f(rgb):
    ''' float rgb to hex'''
    return rgb_to_hex_i(*(np.array(rgb)*255).astype(int))

def hex_to_rgb(value):
    ''' hex to float rgb '''
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i+lv//3], 16)/255 for i in range(0, lv, lv//3))


    

In [37]:
# GUI parts
# type name boxes
w_name_boxes = [widgets.Text(description="", value=types[i], layout={'width': 'initial', 'max_width':'100px'}) for i in range(N_type)]

# type color pickers
w_color_pickers = [widgets.ColorPicker(value = rgb_to_hex_f(palette[i]), concise = True) for i in range(N_type)]

wb_name_color = widgets.VBox(children = [widgets.HBox((w_name_boxes[i], w_color_pickers[i])) for i in range(N_type)])

# base image
w_im = widgets.Image(layout={'max_width': '65%', 
    'max_height': '900px',
    # 'object_fit':'contain'
    })
# core dropdown
w_fn = widgets.Dropdown(options = fn_ls, value = fn_ls[0], description='Core', 
            style={'description_width': 'initial'},
            layout={'width': 'initial', 'max_width':'200px'})

# window size
w_padding = widgets.IntText(description = "radius", value = initial_padding, step=3, min = 0, max=300,
            style={'description_width': 'initial'},
            layout={'width': 'initial', 'max_width':'200px'})

# # marker selection
# w_m = widgets.SelectMultiple(options = markers, value = markers, concise = True, rows = 10, 
#             # style={'description_width': 'initial'},
#             layout={'width': 'initial', 'max_width':'200px'})

# zoomed image preview
w_zoom = widgets.Image(layout={'width': '50%', 
    'max_width':"200px",
    'object_fit':'contain'
    })

# refresh button
w_button_refresh = widgets.Button(description='Refresh', layout={'width': 'initial', 'max_width':'200px'})

# info label
w_l = widgets.Label(value='Ready. Cursor position will be shown here.')

wb_right = widgets.VBox(( 
    w_fn, 
    w_padding, 
    wb_name_color,
    w_button_refresh ,
    w_zoom,
    ))

app = widgets.VBox((widgets.HBox((w_im, wb_right)), w_l))
app.markers = markers

In [38]:
# GUI functions and event listeners
def readpalette():
    colors = [hex_to_rgb(w_c.value) for w_c in w_color_pickers]
    return sns.color_palette(colors)

def nn2Bytes(nn):
    '''converts an image np array to an actual image and save it in the buffer. '''
    img_byte_arr = io.BytesIO()
    plt.imsave(img_byte_arr, nn, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()
    return img_byte_arr

def fig2Bytes():
    '''converts current figure to the image buffer'''
    img_byte_arr = io.BytesIO()
    plt.savefig(img_byte_arr, format='PNG')
    img_byte_arr = img_byte_arr.getvalue()
    return img_byte_arr

def getbaseim(pixeltype_dict = pixeltype_dict):
    nn = pixeltype_dict[w_fn.value]
    palette = readpalette()
    palette.reverse()
    palette.append((0,0,0))
    palette.reverse()
    cmap = ListedColormap(palette)
    '''converts an integer pixel type map array to an RGB image buffer'''
    # convert 2D pixel type integers to 3D rgb with cmap
    w_im.base_im = cmap(nn)
    
    # load into buffer

def getbaseimbuffer(pixeltype_dict = pixeltype_dict):
    getbaseim(pixeltype_dict = pixeltype_dict)
    return nn2Bytes(w_im.base_im)

def zoominregionloc(x, y):
    nn = pixeltype_dict[w_fn.value]
    padding = w_padding.value
    [xmax, ymax] = nn.shape[0:2]

    if 2*padding > min(xmax, ymax):
        padding = int(min(xmax, ymax)-1)/2
    
    if x - padding < 0:
        left, right = 0, 2*padding
    elif x + padding > xmax:
        left, right = xmax - 2*padding, xmax
    else:
        left, right = x - padding, x + padding

    if y - padding < 0:
        top, bot = 0, 2*padding
    elif y + padding > ymax:
        top, bot = ymax - 2*padding, ymax
    else:
        top, bot = y - padding, y + padding
    return left, right, top, bot


def zoominregion(x, y):
    base_im = w_im.base_im
    left, right, top, bot = zoominregionloc(x, y)
    markers = app.markers
    N = len(markers) + 1
    y_n = int(np.ceil(np.sqrt(N)))
    x_n = int(np.ceil(N/y_n))
    fig, axes = plt.subplots(x_n,y_n,figsize=(y_n*2,x_n*2), facecolor=(1,1,1))
    for i in range(x_n):
            for j in range(y_n):
                plt.sca(axes[i][j])
                marker_id = i*y_n + j
                if marker_id == 0:
                    im = base_im
                    plt.imshow(im[left:right, top:bot, :])
                    plt.title('Type')
                    plt.axis('off')
                elif marker_id <= N:
                    m = markers[marker_id - 1]
                    im = singlechannel(w_fn.value, m)
                    plt.imshow(im[left:right, top:bot, :])
                    plt.title(m)
                    plt.axis('off')
                else:
                    axes[i][j].set_visible(False)
    plt.suptitle(f"{w_fn.value}, ({y}, {x}), padding = {w_padding.value}")
    plt.tight_layout()
    
def nozoom():
    fig = plt.figure(facecolor=(0,0,0), figsize=(2,2))
    plt.imshow(np.zeros([1,1,3]))
    plt.axis('off')
    plt.text(0,0, "Hover over the image to show preview here.", ha='center',va='center',wrap=True, color=(1,1,1), fontsize='large')
    w_zoom.value = fig2Bytes()
    plt.close(fig)
    


def showfield(x, y):
    left, right, top, bot = zoominregionloc(x, y)
    base_im = w_im.base_im.copy()


    w_zoom.value = nn2Bytes(base_im[left:right, top:bot, :])

    base_im[left, top:bot, :] = 0
    base_im[right, top:bot, :] = 0
    base_im[left:right, bot, :] = 0
    base_im[left:right, top, :] = 0

    base_im[left-1, top-1:bot+1, :] = 1
    base_im[right+1, top-1:bot+1, :] = 1
    base_im[left-1:right+1, bot+1, :] = 1
    base_im[left-1:right+1, top-1, :] = 1   

    base_im[left+1, top+1:bot-1, :] = 1
    base_im[right-1, top+1:bot-1, :] = 1
    base_im[left+1:right-1, bot-1, :] = 1
    base_im[left+1:right-1, top+1, :] = 1  
    w_im.value = nn2Bytes(base_im)
    
def cbk_click(event):

    x = event['dataX']
    y = event['dataY']
    zoominregion(y, x)
        
    
event_click = Event()
event_click.source = w_im
event_click.watched_events = ['click']
event_click.wait = 0
event_click.on_dom_event(cbk_click)

def refreshbase(*args):
    w_im.value = getbaseimbuffer()

w_button_refresh.on_click(refreshbase)

def cbk_move(event):
    x = event['dataX']
    y = event['dataY']
    if x + y > 0:
        w_l.value = f"({x}, {y})"
        showfield(y, x)
        
event_move = Event()
event_move.source = w_im
event_move.watched_events = ['mousemove']
event_move.wait = refreshrate
# event_move.throttle_or_debounce = 'debounce'
event_move.on_dom_event(cbk_move)

def cbk_leave(event):
    w_l.value = 'Ready. Cursor position will be shown here.'
    nozoom()

event_leave = Event()
event_leave.source = w_im
event_leave.watched_events = ['mouseleave']
event_leave.on_dom_event(cbk_leave)
refreshbase()
nozoom()

In [39]:
display(app)